In [ ]:
import json
from tqdm import tqdm

In [ ]:
big = "memoryalpha"
small = "stexpanded"

mapping_file_big = "./data/triples_v2/" + big + "_mapping.json"
mapping_file_small = "./data/triples_v2/" + small + "_mapping.json"
exact_match_file = "./data/exact_match/" + big + "-" + small + ".json"

communities_big_file = "./data/one_step/" + big + ".csv"
communities_small_file = "./data/one_step/" + small + ".csv"

In [ ]:
with open(exact_match_file) as emf, open(mapping_file_big) as mfb, open(mapping_file_small) as mfs:
    exact_match = json.load(emf)
    mapping_big = json.load(mfb)
    mapping_small = json.load(mfs)

    big_id_dict = {key: value for key, value in mapping_big.items()}
    small_id_dict = {key: value for key, value in mapping_small.items()}
    
    formatted_json = []
    for pair in exact_match:
        big_id = big_id_dict.get(pair[0])
        small_id = small_id_dict.get(pair[1])
        if big_id and small_id:
            formatted_json.append([big_id, small_id])

    # with open('formatted.json', 'w') as outfile:
        # json.dump(formatted_json, outfile, indent=4)
# print(formatted_json)

In [ ]:
in_exact_match_big_file = "./data/one_step_in_exact_match/" + big + ".csv"
in_exact_match_small_file = "./data/one_step_in_exact_match/" + small + ".csv"

big_id_keep = set()
small_id_keep = set()

for pair in formatted_json:
    big_id_keep.add(pair[0])
    small_id_keep.add(pair[1])

with open(communities_big_file, "r") as cbf, open(in_exact_match_big_file, "w") as output:
    for line in cbf:
        numbers = [int(num) for num in line.split(';')]
        for number in numbers:
            if number in big_id_keep:
                output.write(str(number) + ";")
        output.write("\n")

with open(communities_small_file, "r") as csf, open(in_exact_match_small_file, "w") as output:
    for line in csf:
        numbers = [int(num) for num in line.split(';')]
        for number in numbers:
            if number in small_id_keep:
                output.write(str(number) + ";")
        output.write("\n")

In [ ]:
small_communities = []
big_communities = []

with open(in_exact_match_big_file) as cbf, open(in_exact_match_small_file) as csf:
    
    for line in csf:
        numbers_set = {int(num) for num in line.strip().split(';')[:-1]}
        small_communities.append(numbers_set)

    for line in cbf:
        numbers_set = {int(num) for num in line.strip().split(';')[:-1]}
        big_communities.append(numbers_set)

exact_match_count_dict = dict()
for id_big, id_small in formatted_json:
    print(id_big, id_small)
    indexes_big = [index for index, s in enumerate(big_communities) if id_big in s]
    indexes_small = [index for index, s in enumerate(small_communities) if id_small in s]
    print(indexes_big)
    print(indexes_small)
    for i_big in indexes_big:
        for i_small in indexes_small:
            if i_big in exact_match_count_dict:
                exact_match_count_dict[(i_big, i_small)] += 1
            else:
                exact_match_count_dict[(i_big, i_small)] = 1
    print("#############################################")

    # line_num = 0
    # for line in cbf:
    #     line_num += 1
    #     best_match = None
    #     max_match = 0
    #     numbers_set = {int(num) for num in line.strip().split(';')[:-1]}
    #     comm_num = 0
    #     for community in tqdm(small_communities):
    #         comm_num += 1
    #         match_count = 0
    #         for node in community:
    #             for number in numbers_set:
    #                 if [number, node] in formatted_json:
    #                     match_count += 1
    #         if match_count > max_match:
    #             best_match = comm_num
    #             max_match = match_count
    #             print(str(line_num) + " --- " + str(best_match) + " --- " + str(max_match) + "\n")